In [45]:
import os
import numpy as np
from pathlib import Path
import pandas as pd

from sklearn import preprocessing

# Importing libraries for Clustering Algorithms
from kmodes.kprototypes import KPrototypes
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

#importing libraries for display and visualisation of Results
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

#importing libraries for performance measuring
from sklearn.metrics import precision_score, recall_score, f1_score

PATH_ROOT = Path(os.getcwd())
TrainSet_ROOT = os.path.join(PATH_ROOT.parent,'TonysWorkspace/Pre-Processing','ToyCorollaTrainSET.csv')
TestSet_ROOT = os.path.join(PATH_ROOT.parent,'TonysWorkspace/Pre-Processing','ToyCorollaTestSET.csv')

TrainSet = pd.read_csv(TrainSet_ROOT, engine = 'python')
TestSet = pd.read_csv(TestSet_ROOT, engine = 'python')

In [46]:
#dropping artifact from reading csv file
TrainSet.drop('Unnamed: 0', axis=1, inplace=True)
TestSet.drop('Unnamed: 0', axis=1, inplace=True)

Functions:

In [47]:
#this function will encode categorical attributes in the dataset for K-Means and GMM
def DataEncoder(df):
    le = preprocessing.LabelEncoder() 
    columns=['eeca_vehicle_type','eeca_transmission','eeca_fuel', 'eeca_drive']
    for x in columns:
        df[x] = le.fit_transform(df[x].astype(object))
    return df

In [48]:
#this function will remove the attributes that aren't relevent with the clustering process
def DFDrop(df):
    df=df.drop(columns=['eeca_make','eeca_model','VerifiedVariant'])
    return df

In [49]:
#this function's purpose is to fill in null values with the mean of the attribute
def ReplaceNAN(df):
    df = df.fillna(df.mean())
    return df

In [50]:
#this function will label the clusters that was generated by the algorithms
def AssignCluster(df,ClassVari,uniVal):
    df['ClassVari']=ClassVari
    df=df.assign(ClusterAssign='HelloCap')
    for x in range(uniVal):
        ClusterDf=df[df['Cluster']==x]
        MostCommonVal=ClusterDf['ClassVari'].value_counts().idxmax()
        df.loc[df['Cluster'] == x, 'ClusterAssign'] = MostCommonVal
    return df

In [51]:
def kmeans(df,df1,uniVal):
    KmeansModel = KMeans(n_clusters = uniVal, init = 'k-means++', max_iter=100, n_init=10, random_state = None)
    model = KmeansModel.fit(df)
    y_model = KmeansModel.predict(df)
    y1_model = KmeansModel.predict(df1)
    df['Cluster'] = y_model
    df1['Cluster'] = y1_model
    return df,df1

In [52]:
def GMM(df,df1,uniVal):
    GMMmodel = GaussianMixture(n_components= uniVal, # this is the number of clusters
                         covariance_type='tied', # {‘full’, ‘tied’, ‘diag’, ‘spherical’}, default=’full’
                         max_iter=100, # the number of EM iterations to perform. default=100
                         n_init=1, # the number of initializations to perform. default = 1
                         init_params='kmeans', # the method used to initialize the weights, the means and the precisions. {'random' or default='k-means'}
                         verbose=0, # default 0, {0,1,2}
                         random_state=1 # for reproducibility
                        )
    model = GMMmodel.fit(df)
    y_model = GMMmodel.predict(df)
    y1_model = GMMmodel.predict(df1)
    df['Cluster'] = y_model
    df1['Cluster'] = y1_model
    return df,df1

In [53]:
def kprototype(df,df1,uniVal,position):
    PrototypeModel = KPrototypes(n_jobs = -1, n_clusters = uniVal-1, init = 'Cao', max_iter=100, random_state = None, 
                                 n_init = 100)
    model = PrototypeModel.fit(df, categorical = position)
    y_model = PrototypeModel.predict(df, categorical = position)
    y1_model = PrototypeModel.predict(df1, categorical = position)
    df['Cluster'] = y_model
    df1['Cluster'] = y1_model
    return df,df1

In [54]:
# this function will calculate scores based on the results 
def scores(cf, y_true, y_pred, categories, algorithm):

        accuracy  = np.trace(cf) / float(np.sum(cf))

        precision = precision_score(y_true, y_pred, labels=categories, average='macro')
        recall    = recall_score(y_true, y_pred, labels=categories, average='macro')
        f1score  = f1_score(y_true, y_pred, labels=categories, average='macro')
        array = [algorithm, accuracy, precision, recall, f1score]
        return array

In [55]:
# constructing the results table
final_columns = ['Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
final_df = pd.DataFrame(columns = final_columns)

# K-Means Implementation

In [56]:
#Storing the verification data into array for use later and setting number of clusters
ClassVari=TrainSet['VerifiedVariant']
ClassVari2 = TestSet['VerifiedVariant']
uniVal=ClassVari.nunique()
#dropping irrelevent attributes 
Xinput=DFDrop(TrainSet)
Xtest=DFDrop(TestSet)
#Data encoding and NULL replacement
Xinput=DataEncoder(Xinput)
Xinput=ReplaceNAN(Xinput)
Xtest=DataEncoder(Xtest)
Xtest=ReplaceNAN(Xtest)

In [57]:
# Running the algorithm and assigning labels
Xinput, Xtest=kmeans(Xinput,Xtest,uniVal)
Xinput=AssignCluster(Xinput,ClassVari,uniVal)
Xtest=AssignCluster(Xtest,ClassVari2,uniVal)

In [58]:
#labels for score calculation
categories = ['2014–19 Corolla Sedan Petrol Hybrid',
 '2014–19 Corolla Waggon Petrol Hybrid', '2014–19 Corolla Hatch Petrol',
 '2014–19 Corolla Sedan Petrol', '2014–19 Corolla Waggon Petrol',
 '2009–13 Corolla Hatch Petrol', '2009–13 Corolla Wagon Petrol Hybrid']
# confusion matrix data for score calculation
cf_matrix = confusion_matrix(ClassVari, Xinput['ClusterAssign'].tolist())
cf_matrix2= confusion_matrix(ClassVari2, Xtest['ClusterAssign'].tolist())
#score calculation
pushtrain = scores(cf_matrix, ClassVari, Xinput['ClusterAssign'].tolist(), categories,
                   'kmeansTrain')
pushtest = scores(cf_matrix2, ClassVari2, Xtest['ClusterAssign'].tolist(), categories, 
                  'kmeansTest')
# adding to results table
final_df.loc[len(final_df.index)] = pushtrain
final_df.loc[len(final_df.index)] = pushtest

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# GMM Implementation

In [59]:
#Storing the verification data into array for use later and setting number of clusters
ClassVari=TrainSet['VerifiedVariant']
ClassVari2 = TestSet['VerifiedVariant']
uniVal=ClassVari.nunique()
#dropping irrelevent attributes 
Xinput=DFDrop(TrainSet)
Xtest=DFDrop(TestSet)
#Data encoding and NULL replacement
Xinput=DataEncoder(Xinput)
Xinput=ReplaceNAN(Xinput)
Xtest=DataEncoder(Xtest)
Xtest=ReplaceNAN(Xtest)

In [60]:
# Running the algorithm and assigning labels
Xinput, Xtest=GMM(Xinput,Xtest,uniVal)
Xinput=AssignCluster(Xinput,ClassVari,uniVal)
Xtest=AssignCluster(Xtest,ClassVari2,uniVal)

In [61]:
# confusion matrix data for score calculation
cf_matrix = confusion_matrix(ClassVari, Xinput['ClusterAssign'].tolist())
cf_matrix2= confusion_matrix(ClassVari2, Xtest['ClusterAssign'].tolist())
#score calculation
pushtrain = scores(cf_matrix, ClassVari, Xinput['ClusterAssign'].tolist(), categories,
                   'GMMTrain')
pushtest = scores(cf_matrix2, ClassVari2, Xtest['ClusterAssign'].tolist(), categories, 
                  'GMMTest')
# adding to results table
final_df.loc[len(final_df.index)] = pushtrain
final_df.loc[len(final_df.index)] = pushtest

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# K_Prototype Implementation

In [62]:
#Storing the verification data into array for use later and setting number of clusters
ClassVari=TrainSet['VerifiedVariant']
ClassVari2 = TestSet['VerifiedVariant']
uniVal=ClassVari.nunique()
#removing irrelevent data
Xinput=DFDrop(TrainSet)
Xtest=DFDrop(TestSet)
#K-Prototype requires index values for the columns (attributes) that are categorical
catColumnsPos = [Xinput.columns.get_loc(col) for col in list(Xinput.select_dtypes('object').columns)]
#replacing null values
Xinput=ReplaceNAN(Xinput)
Xtest=ReplaceNAN(Xtest)

C:\Users\Tony\AppData\Local\Temp\ipykernel_18016\1645020980.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


In [63]:
#Running the algorithm and assigning clusters.
Xinput, Xtest=kprototype(Xinput,Xtest,uniVal,catColumnsPos)
Xinput=AssignCluster(Xinput,ClassVari,uniVal-1)
Xtest=AssignCluster(Xtest,ClassVari2,uniVal-1)

In [64]:
# confusion matrix data for score calculation
cf_matrix = confusion_matrix(ClassVari, Xinput['ClusterAssign'].tolist())
cf_matrix2= confusion_matrix(ClassVari2, Xtest['ClusterAssign'].tolist())
#score calculation
pushtrain = scores(cf_matrix, ClassVari, Xinput['ClusterAssign'].tolist(), categories,
                   'kprototypeTrain')
pushtest = scores(cf_matrix2, ClassVari2, Xtest['ClusterAssign'].tolist(), categories, 
                  'kprototypeTest')
# adding to results table
final_df.loc[len(final_df.index)] = pushtrain
final_df.loc[len(final_df.index)] = pushtest

D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
final_df.to_csv('Results.csv')